In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

# Задание 1

In [ ]:
def get_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )
    return trainset, testset

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.Dropout2d(p=0.1),
            nn.BatchNorm2d(128),
            nn.AdaptiveMaxPool2d(output_size=1),
            nn.Flatten(),
            nn.Linear(in_features=128, out_features=32),
            nn.ReLU(),
            nn.Dropout1d(0.3),
            nn.LayerNorm(32),
            nn.Linear(in_features=32, out_features=10),
        )

    def forward(self, x: torch.Tensor):
        x = self.layers(x)
        return x

In [ ]:
import wandb


def train_loop(dataloader, model, loss_fn, optimizer, device: str):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * dataloader.batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    wandb.log({"train_loss": train_loss / num_batches})


def test_loop(dataloader, model, loss_fn, device):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = accuracy = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy /= size

    wandb.log({"test_loss": test_loss, "test_accuracy": accuracy})

    print(
        f"Test Error: \n Accuracy: {(100 * accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [ ]:
%wandb login

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shadaevf (shadaevf-rtu-mirea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
device="cuda"

trainset, testset = get_data()

batch_size = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
net = Net()
net.to(device)

loss_fn = nn.CrossEntropyLoss()
params = dict(
  lr = 1e-3,
  weight_decay= 1e-8
)
optimizer = torch.optim.Adam(net.parameters(), **params)

wandb.init(
      # Set the project where this run will be logged
      project="ML2_4_1",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name="experiment_2"
      )

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainloader, net, loss_fn, optimizer, device)
    test_loop(testloader, net, loss_fn, device)

torch.save(net.state_dict(), 'model_weights.pth')
torch.save(optimizer.state_dict(), "optimizer_settings.pth")

wandb.finish()

Epoch 1
-------------------------------
loss: 2.487679  [   64/50000]
loss: 2.317317  [  704/50000]
loss: 2.140590  [ 1344/50000]
loss: 2.192800  [ 1984/50000]
loss: 2.262737  [ 2624/50000]
loss: 2.254214  [ 3264/50000]
loss: 1.992481  [ 3904/50000]
loss: 2.001099  [ 4544/50000]
loss: 1.999909  [ 5184/50000]
loss: 2.014502  [ 5824/50000]
loss: 2.339933  [ 6464/50000]
loss: 2.061233  [ 7104/50000]
loss: 2.118289  [ 7744/50000]
loss: 1.914649  [ 8384/50000]
loss: 2.080012  [ 9024/50000]
loss: 1.905033  [ 9664/50000]
loss: 1.936393  [10304/50000]
loss: 1.978029  [10944/50000]
loss: 2.137023  [11584/50000]
loss: 2.092334  [12224/50000]
loss: 1.947073  [12864/50000]
loss: 1.956836  [13504/50000]
loss: 1.925133  [14144/50000]
loss: 2.002147  [14784/50000]
loss: 2.026057  [15424/50000]
loss: 2.028409  [16064/50000]
loss: 1.893734  [16704/50000]
loss: 1.922664  [17344/50000]
loss: 1.835892  [17984/50000]
loss: 1.983885  [18624/50000]
loss: 1.826490  [19264/50000]
loss: 1.967239  [19904/50000]


test_accuracy,▁▃▄▄▅▆▆▆▇▇▇█▇███▇███
test_loss,█▆▅▅▄▄▃▃▂▃▂▂▂▁▁▁▂▁▁▁
train_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
test_accuracy,0.6994
test_loss,0.84514
train_loss,1.38146


#Задание 2

In [ ]:
from torchvision.models import resnet18, vgg13, mobilenet_v3_small

### ResNet

In [ ]:
resnet_model = resnet18(num_classes=10)

In [ ]:
def get_data(data_dir="./data"):
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )
    return trainset, testset

In [ ]:
device = "cuda"

trainset, testset = get_data()

batch_size = 64
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2
)
net = resnet_model
net.to(device)

loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

wandb.init(
    # Set the project where this run will be logged
    project="ML2_4_2",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="resnet_2",
)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(trainloader, net, loss_fn, optimizer, device)
    test_loop(testloader, net, loss_fn, device)

torch.save(net.state_dict(), "model_weights.pth")
torch.save(optimizer.state_dict(), "optimizer_settings.pth")

wandb.finish()

wandb: Currently logged in as: shadaevf (shadaevf-rtu-mirea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1
-------------------------------
loss: 2.443051  [   64/50000]
loss: 2.058531  [  704/50000]
loss: 2.232089  [ 1344/50000]
loss: 1.791043  [ 1984/50000]
loss: 1.947425  [ 2624/50000]
loss: 1.924714  [ 3264/50000]
loss: 1.971899  [ 3904/50000]
loss: 1.871034  [ 4544/50000]
loss: 1.550727  [ 5184/50000]
loss: 1.650631  [ 5824/50000]
loss: 1.694695  [ 6464/50000]
loss: 1.797446  [ 7104/50000]
loss: 1.586695  [ 7744/50000]
loss: 1.580157  [ 8384/50000]
loss: 1.899321  [ 9024/50000]
loss: 1.731528  [ 9664/50000]
loss: 1.776158  [10304/50000]
loss: 1.696362  [10944/50000]
loss: 1.711059  [11584/50000]
loss: 1.409442  [12224/50000]
loss: 1.575964  [12864/50000]
loss: 1.328331  [13504/50000]
loss: 1.631142  [14144/50000]
loss: 1.634313  [14784/50000]
loss: 1.467301  [15424/50000]
loss: 1.474132  [16064/50000]
loss: 1.353265  [16704/50000]
loss: 1.481780  [17344/50000]
loss: 1.550336  [17984/50000]
loss: 1.305186  [18624/50000]
loss: 1.550796  [19264/50000]
loss: 1.431692  [19904/50000]


test_accuracy,▁▅▆▇▇▇▇██▇██████████
test_loss,█▄▃▂▂▂▂▁▂▃▃▂▃▃▄▄▄▃▄▄
train_loss,█▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,0.8463
test_loss,0.77785
train_loss,0.03115


In [ ]:
wandb.finish()

test_accuracy,▁▅█
test_loss,█▃▁
train_loss,█▄▁
test_accuracy,0.7215
test_loss,0.79843
train_loss,0.87406


### VGG

In [ ]:
vgg_model = vgg13(num_classes=10)

In [ ]:
def get_data(data_dir="./data"):
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )
    return trainset, testset

In [ ]:
device = "cuda"

trainset, testset = get_data()

batch_size = 64
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2
)
net = vgg_model
net.to(device)

loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

wandb.init(
    # Set the project where this run will be logged
    project="ML2_4_2",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="vgg13",
)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(trainloader, net, loss_fn, optimizer, device)
    test_loop(testloader, net, loss_fn, device)

torch.save(net.state_dict(), "model_weights.pth")
torch.save(optimizer.state_dict(), "optimizer_settings.pth")

wandb.finish()

Epoch 1
-------------------------------
loss: 2.316156  [   64/50000]
loss: 2.297116  [  704/50000]
loss: 2.308151  [ 1344/50000]
loss: 2.511289  [ 1984/50000]
loss: 2.296706  [ 2624/50000]
loss: 2.300126  [ 3264/50000]
loss: 2.301713  [ 3904/50000]
loss: 2.309482  [ 4544/50000]
loss: 2.302911  [ 5184/50000]
loss: 2.296366  [ 5824/50000]
loss: 2.282675  [ 6464/50000]
loss: 2.256222  [ 7104/50000]
loss: 2.183543  [ 7744/50000]
loss: 2.275263  [ 8384/50000]
loss: 2.279922  [ 9024/50000]
loss: 2.157320  [ 9664/50000]
loss: 2.099302  [10304/50000]
loss: 1.942390  [10944/50000]
loss: 1.945312  [11584/50000]
loss: 1.874749  [12224/50000]
loss: 1.805930  [12864/50000]
loss: 1.960619  [13504/50000]
loss: 1.871050  [14144/50000]
loss: 1.862402  [14784/50000]
loss: 1.879248  [15424/50000]
loss: 1.891916  [16064/50000]
loss: 1.659115  [16704/50000]
loss: 1.753145  [17344/50000]
loss: 1.919910  [17984/50000]
loss: 1.807973  [18624/50000]
loss: 1.611064  [19264/50000]
loss: 1.756188  [19904/50000]


KeyboardInterrupt: 

In [ ]:
wandb.finish()

test_accuracy,▁▂▆▇▇█████
test_loss,█▇▂▂▁▁▁▁▂▂
train_loss,█▆▅▄▃▃▂▂▁▁
test_accuracy,0.7367
test_loss,0.90388
train_loss,0.30197


### MobileNet

In [ ]:
mobnet_model = mobilenet_v3_small(num_classes=10)

In [ ]:
def get_data(data_dir="./data"):
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )
    return trainset, testset

In [ ]:
device = "cuda"

trainset, testset = get_data()

batch_size = 64
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2
)
net = mobnet_model
net.to(device)

loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

wandb.init(
    # Set the project where this run will be logged
    project="ML2_4_2",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="mobnet1",
)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(trainloader, net, loss_fn, optimizer, device)
    test_loop(testloader, net, loss_fn, device)

torch.save(net.state_dict(), "model_weights.pth")
torch.save(optimizer.state_dict(), "optimizer_settings.pth")

wandb.finish()

Epoch 1
-------------------------------
loss: 2.304287  [   64/50000]
loss: 2.096444  [  704/50000]
loss: 1.980592  [ 1344/50000]
loss: 1.839184  [ 1984/50000]
loss: 1.913953  [ 2624/50000]
loss: 1.772933  [ 3264/50000]
loss: 1.634785  [ 3904/50000]
loss: 1.796628  [ 4544/50000]
loss: 1.656427  [ 5184/50000]
loss: 1.496631  [ 5824/50000]
loss: 1.472347  [ 6464/50000]
loss: 1.685889  [ 7104/50000]
loss: 1.486093  [ 7744/50000]
loss: 1.692029  [ 8384/50000]
loss: 1.411882  [ 9024/50000]
loss: 1.498034  [ 9664/50000]
loss: 1.883375  [10304/50000]
loss: 1.501496  [10944/50000]
loss: 1.557839  [11584/50000]
loss: 1.794960  [12224/50000]
loss: 1.390737  [12864/50000]
loss: 1.143298  [13504/50000]
loss: 1.280218  [14144/50000]
loss: 1.509063  [14784/50000]
loss: 1.343849  [15424/50000]
loss: 1.446296  [16064/50000]
loss: 1.443453  [16704/50000]
loss: 1.482042  [17344/50000]
loss: 1.216291  [17984/50000]
loss: 1.434901  [18624/50000]
loss: 1.585588  [19264/50000]
loss: 1.354360  [19904/50000]


test_accuracy,▁▅▆▇▇▇▇█▇███████████
test_loss,█▄▃▂▂▁▁▁▂▁▁▂▂▂▂▂▂▂▂▂
train_loss,█▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
test_accuracy,0.8396
test_loss,0.69074
train_loss,0.09537
